# Harmonic well

In [ ]:
from uibcdf_test_systems import DoubleWell
from uibcdf_test_systems.simulation import langevin_NVT
from uibcdf_test_systems.analysis import get_probability_density

\begin{equation}
P(x)=\frac{1}{Z_{x}} e^{-\frac{kx^{2}}{2k_{B}T}}
\end{equation}

\begin{equation}
Z_{x} = \int_{-\infty}^{\infty} e^{-\frac{kx^{2}}{2k_{B}T}} dx = \sqrt{\frac{2\pi k_{B}T}{k}} 
\end{equation}

Ya que la integral gaussiana es soluble analíticamente:

\begin{equation}
\int_{-\infty}^{\infty} e^{-a x^{2}} dx = \sqrt{\frac{\pi}{a}} 
\end{equation}

Así que:

\begin{equation}
P(x)=\sqrt{\frac{k}{2\pi k_{B}T}} e^{-\frac{kx^{2}}{2k_{B}T}}
\end{equation}

Podemos calcular la posición promedio:

\begin{equation}
\langle x \rangle = \int_{-\infty}^{\infty} x P(x)dx =
\sqrt{\frac{k}{2\pi k_{B}T}} \int_{-\infty}^{\infty} x e^{-\frac{kx^{2}}{2k_{B}T}} dx = 0
\end{equation}

Porque la integral de un momento impar de la distribución gaussiana es $0$:

\begin{equation}
\int_{-\infty}^{\infty} x^{n} e^{-a x^{2}} dx = 0 \hspace{20pt} n=1,3,5,\dots
\end{equation}

Y por último, con ayuda de la integral del segundo momento de la distribución gaussiana,

\begin{equation}
\int_{-\infty}^{\infty} x^{2} e^{-a x^{2}} dx = \frac{\pi^{1/2}}{2a^{3/2}} 
\end{equation}

podemos calcular la desviación cuadrática estandard:

\begin{equation}
\sigma^{2} = \langle \left(\langle x \rangle- x \right)^{2} \rangle =
\int_{-\infty}^{\infty} {x^{2} P(x)dx} = 
\sqrt{\frac{k}{2\pi k_{B}T}} \int_{-\infty}^{\infty} x^{2} e^{-\frac{kx^{2}}{2k_{B}T}} dx =
\frac{k^{1/2}}{\pi^{1/2}(2k_{B}T)^{1/2}} \frac{(2k_{B}T)^{3/2}\pi^{1/2}}{2k^{3/2}} =
\frac{k_{B}T}{k} 
\end{equation}

Ya vemos entonces que la energía potencial promedio de un oscilador armónico es:

\begin{equation}
<U(x)> = \frac{1}{2}k<x^{2}> = \frac{1}{2} k_{B}T
\end{equation}

Y dado que sabemos que la energía cinética promedio es también:

\begin{equation}
<K(v_{x})> = \frac{1}{2}m<v^{2}> = \frac{1}{2} k_{B}T
\end{equation}

La energía total promedio de un oscilador unidimensional es:

\begin{equation}
<U(x) + K(v_{x})> = k_{B}T
\end{equation}